In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/RAG/'
else:
    path = '../'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
train =  pd.read_csv(path + 'data/'+ 'train_lenplus.csv', index_col  = 0)
test =  pd.read_csv(path + 'data/'+ 'test_one.csv', index_col  = 0)

In [5]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer =  AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### 데이터셋 불러오기


In [9]:
from datasets import DatasetDict, Dataset
train_dataset = DatasetDict()
test_dataset = DatasetDict()


train_dataset = Dataset.from_pandas(train)
test_dataset['test'] = Dataset.from_pandas(test)

In [2]:

# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     global instruction
#     instructions = [instruction for _ in range(6614)]
#     inputs       = examples["사고원인"]
#     outputs      = examples["재발방지대책 및 향후조치계획"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }
#     pass
# dataset_preprocessed = train_dataset.map(formatting_prompts_func, batched = True,)

In [10]:

instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    global instruction
    instructions = [instruction for _ in range(6614)]
    inputs       = examples["사고원인"]
    outputs      = examples["재발방지대책 및 향후조치계획"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        messages = [
            {"role": "system", "content": f"{instruction}"},
            {"role": "user", "content": f"{input}"},
            {"role": "assistant", "content": f"{output}"},
            ]

        text = tokenizer.apply_chat_template(messages, tokenize=False)
        text += '<|end_of_text|>'
        texts.append(text[17:])
    return { "text" : texts, }
    pass
dataset_preprocessed = train_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/6614 [00:00<?, ? examples/s]

In [89]:
# messages = [
#     {"role": "system", "content": f"{instruction}"},
#     {"role": "user", "content": f"{input}"},
#     {"role": "assistant", "content": f"{output}"},
#     ]

# text = tokenizer.apply_chat_template(messages, tokenize=False)
# text += '<|end_of_text|>'

In [23]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [25]:
tokenizer.eos_token

'<|eot_id|>'

In [90]:
# text[17:]

'<|start_header_id|>system<|end_header_id|>\n\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.<|eot_id|><|end_of_text|>'

In [91]:
# EOS_TOKEN

'<|eot_id|>'

In [92]:
# dataset_preprocessed

Dataset({
    features: ['사고원인', '재발방지대책 및 향후조치계획', '__index_level_0__', 'text'],
    num_rows: 6614
})

In [18]:
# instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
# input = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
# output = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'

In [25]:
# messages = [
#     {"role": "system", "content": f"{instruction}"},
#     {"role": "user", "content": f"{input}"},
#     {"role": "assistant", "content": f"{output}"},
#     ]

# text = tokenizer.apply_chat_template(messages, tokenize=False)
# tokenized = tokenizer(text, truncation=True, padding="max_length", max_length=max_seq_length, return_tensors="pt")
# tokenized['input_ids'][0]

tensor(128000)

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_preprocessed,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

In [12]:
from torch.utils.data import DataLoader

dataloader = DataLoader(trainer.train_dataset, batch_size=trainer.args.per_device_train_batch_size, shuffle=True)
# batch = next(iter(dataloader))


In [13]:
tokenizer.decode(trainer.train_dataset['input_ids'][0])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n교각 기초철근 조립 중 강한 바람에 의해 기둥측 주철근이 균형을 잃고 전도되어, 하부에서 작업 중이던 철근공 2명이 철근에 부딪힌 사고<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, 근로자 안전교육을 강화하며, 본 사고와 관련된 유사 피해 발생 방지를 위한 안전교육을 실시할 계획.<|eot_id|><|end_of_text|>'

In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,614 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.787500
2,2.380000
3,2.899000
4,2.115400
5,2.410100
6,2.037800
7,1.998800
8,1.929500
9,2.073900
10,1.818400


In [28]:
instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'
input = test.iloc[3].item()
messages = [
    {"role": "system", "content": f"{instruction}"},
    {"role": "user", "content": f"{input}"},
    {"role": "assistant", "content": ""},
    ]

test_text = tokenizer.apply_chat_template(messages, tokenize=False)

In [29]:
print(test_text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

작업 발판 위 벽돌 잔재를 밟고 넘어짐<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|>


In [31]:

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer([test_text], return_tensors = "pt").to("cuda")
# inputs
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = max_seq_length)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

작업 발판 위 벽돌 잔재를 밟고 넘어짐<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|><|start_header_id|>user<|end_header_id|>

작업발판 위 벽돌 잔재 제거 및 발판 위 안전지대 설치를 통한 작업 방법 전파 및 작업발판 위 안전지대 설치 확인.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

작업발판 위 안전지대 설치 확인, 작업발판 위 안전지대 설치 시 1인 1기계 작업 방지, 작업발판 위 안전지대 설치 시 작업발판 및 작업자 이동 금지, 작업발판 위 안전지대 설치 확인을 통한 작업 전파와 작업발판 위 안전지대 설치 확인을 통한 작업 방법 전파.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

작업발판 위 안전지대 설치 확인, 작업발판 위 안전지대 설치 시 1인 1기계 작업 금지, 작업발판 위 안전지대 설치 시 작업발판 및 작업자 이동 

KeyboardInterrupt: 